In [18]:
import exoplanet as xo
import specgp as sgp
import numpy as np
import theano.tensor as tt
import theano 


logS0 = tt.scalar()
logw0 = tt.scalar()
logQ = tt.scalar()
logS0.tag.test_value = -5.0
logw0.tag.test_value = -2.0
logQ.tag.test_value = 1.0
term = xo.gp.terms.SHOTerm(S0=tt.exp(logS0), w0=tt.exp(logw0), Q=tt.exp(logQ))

alpha = tt.vector()
alpha.tag.test_value = np.array([1, 2, 3])
kernel = sgp.terms.KronTerm(term, alpha=alpha)

t = tt.vector()
diag = tt.dmatrix()
t.tag.test_value = np.linspace(0, 10, 10)
diag.tag.test_value = 1e-5 * tt.ones((3, 10))
mean = tt.dmatrix()
mean.tag.test_value = 0.0 * tt.ones((3, 10))
Q = alpha[:, None]*alpha[None, :]
#gp = xo.gp.GP(kernel=kernel, diag=diag, mean=sgp.means.KronMean(mean), x=t, J=2)
K = tt.slinalg.kron(term.to_dense(t, tt.zeros_like(t)), Q)
#d = tt.vector()
#d.tag.test_value = 1e-5 * np.ones(30)
#K = K + tt.diag(d)
args = [logS0, logw0, logQ, alpha, t]
get_cov = theano.function(args, K)

In [22]:
get_cov(-5, -2, 1, [1, 2, 3], np.linspace(0, 10, 10))

/Users/tgordon/Library/Python/3.7/lib/python/site-packages/theano/tensor/basic.py:6611: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result[diagonal_slice] = x


array([[0.00247875, 0.0049575 , 0.00743626, 0.00245129, 0.00490258,
        0.00735387, 0.00237148, 0.00474296, 0.00711445, 0.0022439 ,
        0.0044878 , 0.0067317 , 0.00207392, 0.00414784, 0.00622175,
        0.00186755, 0.00373509, 0.00560264, 0.00163128, 0.00326255,
        0.00489383, 0.00137191, 0.00274381, 0.00411572, 0.00109637,
        0.00219274, 0.00328911, 0.00081159, 0.00162318, 0.00243478],
       [0.0049575 , 0.00991501, 0.01487251, 0.00490258, 0.00980516,
        0.01470773, 0.00474296, 0.00948593, 0.01422889, 0.0044878 ,
        0.0089756 , 0.0134634 , 0.00414784, 0.00829567, 0.01244351,
        0.00373509, 0.00747019, 0.01120528, 0.00326255, 0.00652511,
        0.00978766, 0.00274381, 0.00548762, 0.00823143, 0.00219274,
        0.00438548, 0.00657822, 0.00162318, 0.00324637, 0.00486955],
       [0.00743626, 0.01487251, 0.02230877, 0.00735387, 0.01470773,
        0.0220616 , 0.00711445, 0.01422889, 0.02134334, 0.0067317 ,
        0.0134634 , 0.02019511, 0.00622175, 0.

In [2]:
x = tt.scalar()
y = tt.scalar()
f = theano.function([x, y], x+y)

In [6]:
a = f(2, 3)
a

array(5.)

In [ ]:
import exoplanet as xo
import specgp as sgp
import numpy as np
import theano.tensor as tt
import theano 



logS0 = tt.scalar()
logw0 = tt.scalar()
logQ = tt.scalar()
logS0.tag.test_value = -5.0
logw0.tag.test_value = -2.0
logQ.tag.test_value = 1.0
kernel = terms.SHOTerm(S0=tt.exp(logS0), w0=tt.exp(logw0), Q=tt.exp(logQ))

kernel = sgp.terms.KronTerm(term, alpha=[1, 2, 3])
t = np.linspace(0, 10, 10)
diag = 1e-5 * np.ones((3, len(t)))
Q = kernel.alpha[:, None]*kernel.alpha[None, :]
gp = xo.gp.GP(kernel=kernel, diag=diag, mean=sgp.means.KronMean(np.zeros_like(diag)), x=t, J=2)
K = tt.slinalg.kron(term.to_dense(t, np.zeros(len(t))), Q).eval()
K = K + np.diag(np.concatenate(diag))

def test_inverse():
    z = np.random.randn(len(t)*3, 1)
    y = np.dot(np.linalg.inv(K), z)
    y = tt.as_tensor_variable(y)
    assert tt.allclose(y, gp.apply_inverse(z))

In [ ]:
x = gp.apply_inverse(z)

In [ ]:
np.random

In [ ]:
tt.nlinalg.matrix_inverse